# Plotly examples

In [ ]:
import numpy as np
import plotly.graph_objs as go

## A simple 1D line plot with error bars

In [ ]:
N = 50
x = np.arange(N)
y = np.random.rand(N)
data = go.Scatter(x=x, y=y,
                  error_y={"type":"data",
                   "array":0.1*np.random.rand(N),
                   "visible":True}
                 )
layout = {"xaxis": {"title": "Some x label [m]"},
          "yaxis": {"title": "A fancy y label [kg]"},
          "title": "A 1D line plot"
}
fig = go.Figure(data=data, layout=layout)
fig

## 1D histogram plot with error bars

In [ ]:
N = 50
x = np.arange(N+1)
y = np.random.rand(N)
e = 0.1*np.random.rand(N)

data = {"x": x,
        "y": y,
        "error_y": {"type":"data",
                    "array":0.1*np.random.rand(N),
                    "visible":True},
        "type": "bar",
        "marker": {"line":{"width":0}},
        "width": np.ediff1d(x)
}

layout = {"xaxis": {"title": "Some x label [m]"},
          "yaxis": {"title": "A fancy y label [kg]"},
          "title": "A 1D histogram plot"
}
fig = go.Figure(data=data, layout=layout)
fig

## 2D image/heatmap with uniformly sized pixels

In [ ]:
N = 100
M = 50
x = np.arange(N+1)
y = np.arange(M+1)
z = np.random.rand(M, N).astype(np.float64)
data = {"type":"heatmap",
        "x":x,
        "y":y,
        "z":z,
        "colorscale":"Viridis",
        "colorbar":{"title":"Counts", "titleside":"right"}
}
layout = {"xaxis": {"title": "X coordinate"},
          "yaxis": {"title": "y coordinate"},
          "title": "A 2D image"
}
fig = go.Figure(data=data, layout=layout)
fig

## 2D filled contour plot

In [ ]:
N = 100
M = 50
xx = np.arange(N, dtype=np.float64)
yy = np.arange(M, dtype=np.float64)
x, y = np.meshgrid(xx, yy)
b = N/20.0
c = M/2.0
r = np.sqrt(((x-c)/b)**2 + ((y-c)/b)**2)
z = np.sin(r)
data = {"type":"contour",
        "x":x[0,:],
        "y":y[:,0],
        "z":z,
        "colorscale":"Viridis",
        "colorbar":{"title":"Counts", "titleside":"right"}
}
layout = {"xaxis": {"title": "x coordinate"},
          "yaxis": {"title": "y coordinate"},
          "title": "A 2D image"
}
fig = go.Figure(data=data, layout=layout)
fig

## 2D image/heatmap with non-uniformly sized pixels

In [ ]:
N = 10
M = 5
x = np.arange(N+1)**2
y = np.arange(M+1)
z = np.random.rand(M, N).astype(np.float64)
data = {"type":"heatmap",
        "x":x,
        "y":y,
        "z":z,
        "colorscale":"Viridis",
        "colorbar":{"title":"Counts", "titleside":"right"}
}
layout = {"xaxis": {"title": "x coordinate"},
          "yaxis": {"title": "y coordinate"},
          "title": "A 2D image with non-equal sized pixels"
}
fig = go.Figure(data=data, layout=layout)
fig

## 2D scatter plot with different symbol sizes

In [ ]:
N = 100
x = np.random.rand(N).astype(np.float64)
y = np.random.rand(N).astype(np.float64)
z = np.random.rand(N).astype(np.float64)
s = 50.0*np.random.rand(N).astype(np.float64)
data = {"type":"scatter",
        "x":x,
        "y":y,
        "mode":"markers",
        "marker":{"size":s,
                  "color":z,
                  "colorscale":'Jet',
                  "showscale":True,
                  "colorbar":{"title":"Counts",
                              "titleside":"right"}
                 }
}
layout = {"xaxis": {"title": "x coordinate"},
          "yaxis": {"title": "y coordinate"},
          "title": "A 2D scatter plot"
}
fig = go.Figure(data=data, layout=layout)
fig

## 2D heatmap with slider through 3D data cube

In [ ]:
from ipywidgets import VBox, IntSlider
 
z = np.random.rand(10, 10, 10)
idx = 0

data = {"type":"heatmap",
        "z":z[:,:,idx],
        "colorscale":"Viridis",
        "colorbar":{"title":"Counts", "titleside":"right"}
}
layout = {"xaxis": {"title": "x coordinate"},
          "yaxis": {"title": "y coordinate"},
          "title": "A 2D image with non-equal sized pixels"
}
fig = go.FigureWidget(data=data, layout=layout)

slider = IntSlider(value=idx, min=0, max=10, step=1,
                   description="Slice index",
                   continuous_update=True,
                   readout=True)

def update_depth(val):
    idx = val["new"]
    fig.data[0].z = z[:, :, idx]

slider.observe(update_depth, names="value")

box = VBox([fig, slider])
box

## 3D line plot

In [ ]:
N = 100
M = 10
xx = np.arange(N, dtype=np.float64)
yy = np.arange(M, dtype=np.float64)
x, y = np.meshgrid(xx, yy)
b = M/2.0
c = N/2.0
r = np.sqrt(((x-c)/b)**2 + ((y-c)/b)**2)
z = np.sin(r)

layout = {"title": "A 3D scatter plot"}

trace = []
for i in range(M):
    trace.append(go.Scatter3d(x=xx, y=[i]*N, z=z[i, :], mode='lines', line={"width":4}))

fig = go.Figure(data=trace, layout=layout)
fig.update_layout(scene=dict(
    xaxis_title="X coord",
    yaxis_title="Y coord",
    zaxis_title="Z coord"))
fig

## 3D scatter plot

In [ ]:
N = 100
x = np.random.rand(N).astype(np.float64)
y = np.random.rand(N).astype(np.float64)
z = np.random.rand(N).astype(np.float64)
s = 30.0*np.random.rand(N).astype(np.float64)
c = np.abs(z)
trace = go.Scatter3d(x=x, y=y, z=z,
                     mode="markers",
                     marker={"size":s,
                               "color":z,
                               "colorscale":'Jet',
                               "showscale":True,
                               "colorbar":{"title":"Counts",
                                           "titleside":"right"}
                              }
                    )

fig = go.Figure(data=trace)
fig.update_layout(scene=dict(
    xaxis_title="X coord",
    yaxis_title="Y coord",
    zaxis_title="Z coord"))
fig

## 3d surface

In [ ]:
X = np.arange(-5, 5, 0.25)
Y = np.arange(-5, 5, 0.25)
X, Y = np.meshgrid(X, Y)
R = np.sqrt(X**2 + Y**2)
Z = np.sin(R)

surf = go.Surface(x=X, y=Y, z=Z)

fig = go.Figure(data=surf)
fig.update_layout(scene=dict(
    xaxis_title="X coord",
    yaxis_title="Y coord",
    zaxis_title="Z coord"))
fig

## 3D slicer

In [ ]:
from matplotlib.widgets import Slider

N = 50
M = 40
L = 30
xx = np.arange(N, dtype=np.float64)
yy = np.arange(M, dtype=np.float64)
zz = np.arange(L, dtype=np.float64)
x, y, z = np.meshgrid(xx, yy, zz, indexing='ij')
b = N/20.0
c = M/2.0
d = L/2.0
r = np.sqrt(((x-c)/b)**2 + ((y-c)/b)**2 + ((z-d)/b)**2)
a = np.sin(r)
X, Y = np.meshgrid(xx, yy, indexing='ij')

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
fig.subplots_adjust(bottom=0.15)

color_dimension = a[:,:,idx] # change to desired fourth dimension
minn, maxx = color_dimension.min(), color_dimension.max()
from matplotlib.colors import Normalize
norm = Normalize(minn, maxx)
m = plt.cm.ScalarMappable(norm=norm, cmap='viridis')
m.set_array([])
fcolors = m.to_rgba(color_dimension)

# im_h = ax.imshow(data[:, :, idx], interpolation='nearest')
surf = ax.plot_surface(X, Y, np.zeros_like(X), linewidth=0, antialiased=False, facecolors=fcolors)

ax.set_xlabel("x coordinate")
ax.set_ylabel("y coordinate")
ax.set_title("3D slicer")
ax.set_zlim([0, L])
cb = plt.colorbar(surf)
cb.ax.set_ylabel("Counts")

ax_depth = plt.axes([0.23, 0.02, 0.56, 0.04])
slider_depth = Slider(ax_depth,
    'depth',
    0,
    L-1,
    valinit=idx)

def update_depth(val):
    idx = int(round(slider_depth.val))
    fcolors = m.to_rgba(a[:,:,idx])
#     surf.set_verts() # Could not get the slider to work
  
slider_depth.on_changed(update_depth)
  
plt.show()

## Additional examples
<a id='interactive_examples'></a>
### Interactive 1: Scrolling with the mouse through the 3rd dimension of a 3D data cube

Source: https://matplotlib.org/examples/pylab_examples/image_slices_viewer.html

In [ ]:
class IndexTracker(object):
    def __init__(self, ax, X):
        self.ax = ax
        ax.set_title('use scroll wheel to navigate images')

        self.X = X
        rows, cols, self.slices = X.shape
        self.ind = self.slices//2

        self.im = ax.imshow(self.X[:, :, self.ind])
        self.update()

    def onscroll(self, event):
        print("%s %s" % (event.button, event.step))
        if event.button == 'up':
            self.ind = np.clip(self.ind + 1, 0, self.slices - 1)
        else:
            self.ind = np.clip(self.ind - 1, 0, self.slices - 1)
        self.update()

    def update(self):
        self.im.set_data(self.X[:, :, self.ind])
        ax.set_ylabel('slice %s' % self.ind)
        self.im.axes.figure.canvas.draw()


fig, ax = plt.subplots(1, 1)

X = np.random.rand(20, 20, 40)

tracker = IndexTracker(ax, X)


fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()

## Interactive 2: Click on a legend item to hide/show the corresponding line

Source: https://matplotlib.org/examples/event_handling/legend_picking.html

In [ ]:
t = np.arange(0.0, 0.2, 0.1)
y1 = 2*np.sin(2*np.pi*t)
y2 = 4*np.sin(2*np.pi*2*t)

fig, ax = plt.subplots()
ax.set_title('Click on legend line to toggle line on/off')
line1, = ax.plot(t, y1, lw=2, color='red', label='1 HZ')
line2, = ax.plot(t, y2, lw=2, color='blue', label='2 HZ')
leg = ax.legend(loc='upper left', fancybox=True, shadow=True)
leg.get_frame().set_alpha(0.4)


# we will set up a dict mapping legend line to orig line, and enable
# picking on the legend line
lines = [line1, line2]
lined = dict()
for legline, origline in zip(leg.get_lines(), lines):
    legline.set_picker(5)  # 5 pts tolerance
    lined[legline] = origline


def onpick(event):
    # on the pick event, find the orig line corresponding to the
    # legend proxy line, and toggle the visibility
    legline = event.artist
    origline = lined[legline]
    vis = not origline.get_visible()
    origline.set_visible(vis)
    # Change the alpha on the line in the legend so we can see what lines
    # have been toggled
    if vis:
        legline.set_alpha(1.0)
    else:
        legline.set_alpha(0.2)
    fig.canvas.draw()

fig.canvas.mpl_connect('pick_event', onpick)

plt.show()

## 3. Slider and RadioButtons demo

Source: https://matplotlib.org/3.1.1/gallery/widgets/slider_demo.html

In [ ]:
from matplotlib.widgets import Slider, Button, RadioButtons

fig, ax = plt.subplots()
plt.subplots_adjust(left=0.25, bottom=0.25)
t = np.arange(0.0, 1.0, 0.001)
a0 = 5
f0 = 3
delta_f = 5.0
s = a0 * np.sin(2 * np.pi * f0 * t)
l, = plt.plot(t, s, lw=2)
ax.margins(x=0)

axcolor = 'lightgoldenrodyellow'
axfreq = plt.axes([0.25, 0.1, 0.65, 0.03], facecolor=axcolor)
axamp = plt.axes([0.25, 0.15, 0.65, 0.03], facecolor=axcolor)

sfreq = Slider(axfreq, 'Freq', 0.1, 30.0, valinit=f0, valstep=delta_f)
samp = Slider(axamp, 'Amp', 0.1, 10.0, valinit=a0)


def update(val):
    amp = samp.val
    freq = sfreq.val
    l.set_ydata(amp*np.sin(2*np.pi*freq*t))
    fig.canvas.draw_idle()


sfreq.on_changed(update)
samp.on_changed(update)

resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')


def reset(event):
    sfreq.reset()
    samp.reset()
button.on_clicked(reset)

rax = plt.axes([0.025, 0.5, 0.15, 0.15], facecolor=axcolor)
radio = RadioButtons(rax, ('red', 'blue', 'green'), active=0)


def colorfunc(label):
    l.set_color(label)
    fig.canvas.draw_idle()
radio.on_clicked(colorfunc)

plt.show()

<a id='plotly_in_qt'></a>
## Embed plotly graph in Qt window

In [ ]:
import os, sys
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QUrl
from PyQt5 import QtWebEngineWidgets
import plotly.io as pio
import plotly.offline as offplt
import plotly.graph_objs as go


class PlotlyViewer(QtWebEngineWidgets.QWebEngineView):
    def __init__(self, fig, exec=True):
        # Create a QApplication instance or use the existing one if it exists
        self.app = QApplication.instance() if QApplication.instance() else QApplication(sys.argv)

        super().__init__()

        self.file_path = os.path.abspath(os.path.join(os.path.dirname(__file__), "temp.html"))
        html = offplt.plot(fig, output_type='', auto_open=False, include_plotlyjs="True")
        print(html)
        self.setHtml(html)
        self.setWindowTitle("Plotly Viewer")
        self.show()

        if exec:
            self.app.exec_()

    def closeEvent(self, event):
        os.remove(self.file_path)


# Or a custom external renderer
class MyExternalRenderer(pio.base_renderers.ExternalRenderer):
    def activate(self):
        pass

    def render(self, fig_dict):
        for key, fig in fig_dict.items():
            win = PlotlyViewer(fig)


# Register for use by name
my_renderer = MyExternalRenderer()
pio.renderers['my_renderer'] = my_renderer

fig = go.Figure()
fig.add_scatter(x=np.random.rand(100), y=np.random.rand(100), mode='markers',
                marker={'size': 30, 'color': np.random.rand(100), 'opacity': 0.6,
                        'colorscale': 'Viridis'})
fig.show(renderer='my_renderer')